# Proyecto Final 

### Asignatura: 
        Portafolios de Inversión 

### Integrantes: 
        1) Iván Andrés Arellano (714383)
        2) Fernando Aguilar Carvajal (IF724031)
        3) xxxx_xx


In [1]:
# Se importan librerías 
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import scipy.optimize as opt
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader.data as web
from plotly.subplots import make_subplots

In [2]:
# Funciones: 

# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

# Función Varianza
def varianza(w,Sigma):
    return w.T.dot(Sigma).dot(w) 

# Función objetivo
def menos_RS(w,Eind,Sigma,rf):
    Ep = Eind.T.dot(w)
    sp = (w.T.dot(Sigma).dot(w))**.5
    RS = (Ep-rf)/sp
    return -RS

# Función Cambio Porcentual
def pct_chng(VA: 'Valor Actual',
             VI: 'Valor Inicial'):
    
    ret = np.round(((VA-VI)/VI), 2)
    return ret


def simulacion_Portafolio(ret: 'Retornos',
                       rf: 'Tasa Libre de Riesgo',
                       g: 'Grado de Aversión al Riesgo'):
    
    global pesos, summary, w_EMV, w_opt, E_EMV, s_EMV,tickers
        
    #Rendimientos y volatilidades esperadas
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                 'Vol':((252)**(.5))*ret.std()})
        
    cov_ret = ret.cov() # Matriz de varianza - covarianza
    
    corr = ret.corr() # Matriz de correlación
    
    S = np.diag(summary['Vol'].values)
    Sigma = S.dot(corr).dot(S)
    Eind = summary['Media'].values
    
    n = len(Eind) # Número de activos    
    w0 = np.ones((n,))/n # Dato inicial   
    bnds = ((0, 1),)*n # Cotas de las variables    
    cons={'type':'eq','fun':lambda w:w.sum()-1} # Restricciones
    
    # Portafolio de mínima varianza
    minvar = minimize(fun=varianza,
                      x0=w0,
                      args=(Sigma,),
                      bounds=bnds,
                      constraints=cons,
                      tol=1e-5)
    
    w_minvar = minvar.x
    
    E_minvar = Eind.T.dot(w_minvar)
    s_minvar = (w_minvar.T.dot(Sigma).dot(w_minvar))**.5
    RS_minvar = (E_minvar - rf)/s_minvar

    # Portafolio EMV
    EMV = minimize(fun = menos_RS,
                   x0 = w0,
                   args = (Eind, Sigma, rf),
                   bounds = bnds,
                   constraints = cons,
                   tol = 1e-5)
    w_EMV = EMV.x
    
    # Pesos, rendimiento y riesgo del portafolio EMV
    E_EMV = Eind.T.dot(w_EMV)
    s_EMV = (w_EMV.T.dot(Sigma).dot(w_EMV))**0.5
    RS_EMV = (E_EMV - rf)/s_EMV
    
    #Asignación de capital
    w_opt = (E_EMV - rf)/(g*s_EMV**2)
    
    pesos = [w_EMV[i]*w_opt for i in range(len(w_EMV))]
    pesos.append(1-w_opt)
    pesos = np.round(pesos, 4)
    
    tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
              'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
              'BIMBOA.MX','GAPB.MX','C.MX',
              'AEROMEX.MX','WALMEX.MX','SAN.MX',
                'GMEXICOB.MX', 'Cetes']
    
    a_c = pd.DataFrame(columns = ['Asignación de Capital'], index=[tickers])
    a_c['Asignación de Capital'] = pesos
    
    return a_c



In [3]:
ETFS = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX','VOX.MX']

Stoks = ['BBVA.MX','AC.MX','AMXL.MX',
         'BIMBOA.MX','CUERVO.MX','VOLARA.MX',
         'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX']

tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
           'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
           'BIMBOA.MX','GAPB.MX','C.MX',
           'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX' ]

## Datos Historicos previos al 2015-12-31

In [4]:
start_date = '2011-12-31'
end_date = '2016-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [68]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port1 = pesos
#Port_prev
W_port1[:-1].sum()

1.5296

In [93]:
def inversion(W:'Pesos',Costo_I:'Costo_Inicial',Costo_F:'Costo Final',capital1):
    DestinadoXaccion = np.floor(W*capital1)
    N_acciones = np.floor(DestinadoXaccion/Costo_I)
    saldo_i = np.array(N_acciones)*Costo_I
    Saldo_Inicial = saldo_i.sum()
    Saldo_f = np.array(N_acciones)*Costo_F
    Saldo_Final = Saldo_f.sum()
    Retorno = ((Saldo_Final/Saldo_Inicial)-1)*100
    
    
    
    return print('Con una inversión de: ', capital1 ,'Un apalancamiento de: ', Saldo_Inicial-capital , 
                 'Tendrías una ganancia de :',Retorno,'% Con un saldo final de:',Saldo_Final)

In [94]:
inversion(w_1,Precio_i,Precio_f,100000)

Con una inversión de:  100000 Un apalancamiento de:  50666.23429107666 Tendrías una ganancia de : 38.799302311128955 % Con un saldo final de: 209123.68201446533


## 2016 

In [13]:
# Portafolio 2016
start_date = '2015-12-31'
end_date = '2016-12-31'
closes_16 = get_adj_closes(tickers=tickers[:-1],
                        start_date= start_date , end_date=end_date)
ret_16 = closes_16.pct_change().dropna()

In [64]:
a =closes_16[:1].sum()
a.sum()

8165.668488502502

In [58]:
w_1 =W_port1[:-1]
capital = 100000
Ret = ret_16
Precio_i = closes_16.iloc[1]
Precio_f = closes_16.iloc[-1]
Dest = np.floor(w_1*capital)
n_Acciones = Dest/Precio_i
n_Acciones

Symbols
XLK.MX           1.577392
VOO.MX           0.400789
VGT.MX           8.542195
VHT.MX           5.452311
VOX.MX          10.031424
BBVA.MX          0.000000
AC.MX            0.000000
AMXL.MX          0.000000
BIMBOA.MX        0.000000
GAPB.MX        701.831200
C.MX            66.375147
AEROMEX.MX     292.252616
WALMEX.MX        0.000000
SAN.MX          13.998816
GMEXICOB.MX      0.000000
Name: 2016-01-04 00:00:00, dtype: float64

In [59]:
N_acciones = np.floor(n_Acciones)
Precios = N_acciones*Precio_i
a =np.sum(Precios)
Saldo_i = a.sum()

In [60]:
a

150666.23429107666

In [54]:
Precios_Final = N_acciones*Precio_f
Saldo_f = Precios_Final.sum()

In [55]:
(Saldo_f/Saldo_i)-1

0.3879930231112896

In [7]:
closes_16.tail()

Symbols,XLK.MX,VOO.MX,VGT.MX,VHT.MX,VOX.MX,BBVA.MX,AC.MX,AMXL.MX,BIMBOA.MX,GAPB.MX,C.MX,AEROMEX.MX,WALMEX.MX,SAN.MX,GMEXICOB.MX,Cetes
Date,,,,,,,,,,,,,,,,
2016-12-26,1009.52002,NaN,NaN,NaN,NaN,138.163269,91.983765,11.295853,41.995285,130.713638,1288.974121,39.209999,32.701839,49.488346,44.403854,NaN
2016-12-27,1025.00000,4317.299805,123.760002,2657.899902,101.400002,138.163269,92.139709,11.330745,41.657745,130.596603,1263.266113,39.150002,32.640274,50.095268,44.523281,NaN
2016-12-28,1020.00000,4284.430176,122.589996,127.029999,100.610001,136.182404,93.352524,11.339468,42.351574,130.822830,1241.874512,39.250000,32.860161,48.802044,45.972359,NaN
2016-12-29,1004.00000,4220.000000,2534.840088,2633.270020,100.779999,136.281464,94.435410,11.452861,44.011131,132.328720,1224.052856,39.230000,33.027271,48.993454,45.948475,NaN
2016-12-30,1010.00000,4252.000000,2534.840088,126.650002,100.000000,137.668060,93.621078,11.365635,44.076763,133.015335,1225.654175,39.099998,32.587494,49.815147,44.929352,NaN


In [20]:
ret_16 = closes_16.pct_change().dropna()